In [20]:
#Load train, test and dev data as list of tuples
def load_data(file_path, mode):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        index, words, tags = [], [], []
        maxlen = 0
        for line in f:
            line = line.strip()
            if not line:
                if words:
                    data.append((index, words, tags))
                    maxlen = max(maxlen, len(words))
                    index, words, tags = [], [], []
            else:
                if mode == 'train' or mode == 'dev':
                  ind, word, tag = line.split(' ')[0], line.split(' ')[1], line.split(' ')[2]
                elif mode == 'test':
                  ind, word = line.split(' ')[0], line.split(' ')[1]

                if mode == 'test':
                  index.append(ind)
                  words.append(word)
                if mode == 'train' or mode == 'dev':
                  index.append(ind)
                  words.append(word)
                  tags.append(tag)
                
    print(f"max sentence length = {maxlen}")
    return data

train_data = load_data('data/train', mode = 'train')
dev_data = load_data('data/dev', mode = 'dev')
test_data = load_data('/data/test', mode = 'test')

max sentence length = 113
max sentence length = 109
max sentence length = 124


In [21]:
# Experiment: Skip learning from sentence that has only O tags to ameliorate class imbalance
# Observation: Might work -> Can't vet reliability

# from collections import Counter
# from copy import deepcopy

# def augment_data(data):
#     count_only_O = 0
#     count_one_other = 0
#     for i, outer_tuple in enumerate(data):
#         tgs = outer_tuple[2]
#         tg_counter = Counter(tg for tg in tgs)
#         keys = list(tg_counter.keys())
#         if len(keys) == 1 and keys[0] == 'O':
#             count_only_O += 1
#             data.remove(outer_tuple)
#         else:
#             count_one_other += 1
#     print(f"Only O count: {count_only_O}")
#     print(f"One other O count: {count_one_other}")
#     return data
    
# augmented_train_data = augment_data(deepcopy(train_data))
# augmented_dev_data = augment_data(deepcopy(dev_data))

In [56]:
# Build vocabulary and word<->tag maps 

from collections import Counter
def build_vocab(data):
    word_counts = Counter(word for _, sentence, _ in data for word in sentence)
    filtered_dict = {key: value for key, value in word_counts.items()}
    vocabulary = ['<pad>', '<unk>'] + sorted(filtered_dict)
    word2idx = {word: idx for idx, word in enumerate(vocabulary)}
    return vocabulary, word2idx

def build_tag_map(data):
    tags = set(tag for _,_, tags in data for tag in tags)
    # tags.add('<pad>')
    tag2idx = {tag: idx for idx, tag in enumerate(sorted(tags))}
    return tag2idx

vocabulary, word2idx = build_vocab(train_data + dev_data + test_data)
tag2idx = build_tag_map(train_data + dev_data)

In [57]:
# Make a reverse word2idx -> idx2word. Needed for decoding after forward pass.
# reverse_word2idx = {v: k for k, v in word2idx.items()}

In [250]:
test_data[0]

(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'],
 ['SOCCER',
  '-',
  'JAPAN',
  'GET',
  'LUCKY',
  'WIN',
  ',',
  'CHINA',
  'IN',
  'SURPRISE',
  'DEFEAT',
  '.'],
 [])

In [268]:
#Make a wrapper class for our Dataset -- Take care of no tag in test data

import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from copy import deepcopy

class NERDataset(Dataset):
    def __init__(self, data, word2idx, tag2idx, mode = 'train', test_word2idx_untouched=None):
        self.data = data
        self.word2idx = word2idx
        self.tag2idx = tag2idx
        self.maxlen=125
        self.mode = mode

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        mode = self.mode
        if mode == 'train' or mode == 'devtrain' or mode == 'dev_out' or mode == 'dev_perl':
          index, words, tags = self.data[idx]
        if mode == 'test_out':
          index, words, _ = self.data[idx]

        
        # print(words)
        temp = [self.word2idx.get(word, self.word2idx['<unk>']) for word in words]
        if len(temp) < self.maxlen:
            pad_arr = np.pad(temp, (0, self.maxlen-len(temp)), 'constant', constant_values=(-1, 0))
        elif len(temp) > self.maxlen:
            pad_arr = temp[:self.maxlen]
        else:
            pad_arr = temp
        x = torch.tensor(pad_arr)

        if mode == 'test_out' or mode =='dev_out' or mode == 'dev_perl':
#           words_untouched = deepcopy(words)
#           print(word)
          temp = [test_word2idx_untouched.get(word, -1) for word in words]
          # print(index)
          if len(temp) < self.maxlen:
              pad_arr = np.pad(temp, (0, self.maxlen-len(temp)), 'constant', constant_values=(-1, 0))
          elif len(temp) > self.maxlen:
              pad_arr = temp[:self.maxlen]
          else:
              pad_arr = temp
          k = torch.tensor(pad_arr)

        # print(words)
        # print(index)
        temp = [int(ind) for ind in index]
        if len(temp) < self.maxlen:
            pad_arr = np.pad(temp, (0, self.maxlen-len(temp)), 'constant', constant_values=(-1, 0))
        elif len(temp) > self.maxlen:
            pad_arr = temp[:self.maxlen]
        else:
            pad_arr = temp
        z = torch.tensor(pad_arr)

        if mode == 'train' or mode == 'devtrain' or mode =="dev_out" or mode == 'dev_perl':
          temp = [self.tag2idx[tag] for tag in tags]
          if len(temp) < self.maxlen:
              pad_arr = np.pad(temp, (0,self.maxlen-len(temp)), 'constant', constant_values=(-1, -1))
          elif len(temp) > self.maxlen:
              pad_arr = temp[:self.maxlen]
          else:
              pad_arr = temp
          y = torch.LongTensor(pad_arr)
        
        if mode == 'train' or mode == 'devtrain':
          return x, y
        # print(x, z, k)
        if mode == 'test_out' or mode == 'dev_out':
            return x, k, z
        if mode == 'dev_perl':
            return x, y, z, k


train_dataset = NERDataset(train_data, word2idx, tag2idx, mode = 'train')
dev_dataset = NERDataset(dev_data, word2idx, tag2idx, mode = 'devtrain')

batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)

In [59]:
tag2idx

{'B-LOC': 0,
 'B-MISC': 1,
 'B-ORG': 2,
 'B-PER': 3,
 'I-LOC': 4,
 'I-MISC': 5,
 'I-ORG': 6,
 'I-PER': 7,
 'O': 8}

In [60]:
#Define Model

import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.init as init


class BLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout, pretrained_embedding = None):
        super(BLSTM, self).__init__()
        if pretrained_embedding:
            self.embedding = pretrained_embedding
        else:
            self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
#         init.xavier_uniform_(self.embedding.weight)
        self.blstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=1, bidirectional=True, batch_first=True)
        self.dropout1 = nn.Dropout(dropout)
        self.linear1 = nn.Linear(hidden_dim * 2, 128)
        self.activation = nn.ELU()
        self.classifier = nn.Sequential(
            nn.Linear(128, 256),
            nn.Tanh(),
            nn.Dropout(0.2),
            nn.Linear(256, output_dim)
        )

    def forward(self, x):
        # print(x.shape)
        embedded = self.embedding(x)
        seq_lengths = torch.count_nonzero(x, dim=1).cpu()
        x = pack_padded_sequence(embedded, seq_lengths, batch_first=True, enforce_sorted=False)
        x, _ = self.blstm(x)
        x, unpacked_len = pad_packed_sequence(x, batch_first=True)
        x = self.dropout1(x)
        x = self.linear1(x)
        x = self.activation(x)
        x = self.classifier(x)
        return x

In [61]:
!pip install torchmetrics

In [62]:
# piecewise accuracy 
def accuracy(outputs, labels):
    acc = 0
    count = 0
    for i in range(outputs.shape[0]):
        sentence_pred = outputs[i]
        for j, word in enumerate(sentence_pred):
            word_pred = torch.argmax(word).item()
            label = labels[i][j].item()
            if label == -1:
                continue
            count += 1
            if word_pred == label:
                acc += 1
    return acc/count

#evaluate function for dev test during training
def evaluate(model, criterion, dataloader, device = 'cuda'):
  with torch.no_grad():
    dev_loss, dev_acc, dev_f1 = 0.0, 0.0, 0.0
    for batch_x, batch_y in tqdm(dataloader):
      batch_x = batch_x.to(device)
      batch_y = batch_y.to(device)
      outputs = model(batch_x)
      seq_lengths = torch.count_nonzero(batch_x, dim=1).to('cpu')
      packed_y = pack_padded_sequence(batch_y, seq_lengths, batch_first=True, enforce_sorted=False)
      unpacked_y, unpacked_len = pad_packed_sequence(packed_y, batch_first=True, padding_value=-1)
      unpacked_y = unpacked_y.to(device)
      loss = criterion(outputs.permute(0, 2, 1), unpacked_y)
      dev_loss += loss.item()
      out_for_f1 = torch.argmax(outputs, dim = -1)
      mask = (unpacked_y >= 0)
      f1 = f1_score(out_for_f1[mask].cpu(), unpacked_y[mask].cpu(), average='weighted')
      #-->costly operation, uncomment to see accuracy
#       acc = accuracy(outputs, batch_y) 
#       dev_acc += acc
      dev_f1 += f1
    dev_loss /= len(dataloader)
    dev_acc /= len(dataloader)
    dev_f1 /= len(dataloader)

    print(f"Average Dev Loss: {dev_loss}")
    print(f"Average Dev accuracy: {dev_acc}")
    print(f"Average Dev F1: {dev_f1}")
    return dev_loss, dev_f1


In [63]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [64]:
from sklearn.utils.class_weight import compute_class_weight 

def get_class_weights(data):
    all_y = []
    for data in data:
        all_y.extend(data[2])

    class_weights = compute_class_weight(
                                            class_weight = "balanced",
                                            classes = np.unique(all_y),
                                            y = all_y                                                    
                                        )
    class_weights=torch.tensor(class_weights,dtype=torch.float).to('cuda')
    return class_weights

class_weights = get_class_weights(train_data + dev_data)

In [65]:
# class_weights -= torch.min(class_weights)
class_weights

tensor([ 3.1704,  6.5276,  3.7145,  3.3713, 20.1275, 18.9609,  6.3884,  4.8775,
         0.1333], device='cuda:0')

In [66]:
from tqdm import tqdm
from sklearn.metrics import f1_score
from torchmetrics.functional.classification import multiclass_f1_score
from torch.optim.lr_scheduler import MultiStepLR


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BLSTM(len(vocabulary), embedding_dim=100, hidden_dim=256, output_dim=len(tag2idx), dropout=0.33).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, weight_decay = 1e-5, eps=1e-08, betas = (0.9, 0.999))
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9, weight_decay = 1e-3)
# scheduler = MultiStepLR(optimizer, milestones=[3,5,7,9], gamma=0.1)
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.5, max_lr=1.2, step_size_up=20, step_size_down=None, mode='triangular', gamma=1.0)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', min_lr=1, verbose=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',factor=0.55, patience = 3, threshold=0.1, verbose=True, min_lr=5e-4)
# scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.5, total_iters=20)

# class_weights = torch.tensor([1,1,1,1,1,1,1,1,0.01], dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=-1, reduction='mean', weight=class_weights)
    

def train(model, train_loader, optimizer, criterion, device, epochs):
    model.train()
    SAVE_PATH = "./best_model.pt"
    best_f1 = -1
    for epoch in range(epochs):
      print(f"Epoch: {epoch}")
      train_loss, train_acc, train_f1 = 0.0, 0.0, 0.0
      for batch_x, batch_y in tqdm(train_loader):
          batch_x = batch_x.to(device)
          batch_y = batch_y.to(device)
          outputs = model(batch_x)

          seq_lengths = torch.count_nonzero(batch_x, dim=1).to('cpu')
          packed_y = pack_padded_sequence(batch_y, seq_lengths, batch_first=True, enforce_sorted=False)
          unpacked_y, unpacked_len = pad_packed_sequence(packed_y, batch_first=True, padding_value = -1)
          unpacked_y = unpacked_y.to(device)
            
          loss = criterion(outputs.permute(0, 2, 1), unpacked_y)
          train_loss += loss.item()
  
#           mask = (unpacked_y >= 0) -->costly operation, uncomment to see accuracy, f1
#           acc = accuracy(outputs, batch_y)
#           out_for_f1 = torch.argmax(outputs, dim = -1)
#           f1 = f1_score(out_for_f1[mask].cpu(), unpacked_y[mask].cpu(), average='weighted')
#           train_acc += acc
#           train_f1 += f1

          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          # print(get_lr(optimizer))

      train_loss /= len(train_loader)
#       train_acc /= len(train_loader)
#       train_f1 /= len(train_loader)
      
#       print(f"Average train accuracy: {train_acc}")
#       print(f"Average train f1: {train_f1}")
      val_loss, val_f1 = evaluate(model, criterion, dev_loader)
      if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), SAVE_PATH)
      scheduler.step(val_loss)
      print(f"Average train Loss: {train_loss}")
      print(f"Current Learning Rate: {get_lr(optimizer)}")
      print(f"Best sklearn masked F1: {best_f1}")

    return model

model = train(model, train_loader, optimizer, criterion, device, epochs = 100)


Epoch: 0


100%|██████████| 434/434 [00:02<00:00, 164.55it/s]


Average Dev Loss: 1.4999754266804814
Average Dev accuracy: 0.0
Average Dev F1: 0.44007233480329083
Average train Loss: 1.7876404129135697
Current Learning Rate: 0.01
Best sklearn masked F1: 0.44007233480329083
Epoch: 1


100%|██████████| 434/434 [00:02<00:00, 168.88it/s]


Average Dev Loss: 1.2480844843634813
Average Dev accuracy: 0.0
Average Dev F1: 0.512941720119775
Average train Loss: 1.3169132836854827
Current Learning Rate: 0.01
Best sklearn masked F1: 0.512941720119775
Epoch: 2


...

Average Dev Loss: 0.5446643792452239
Average Dev accuracy: 0.0
Average Dev F1: 0.9574796125732961
Average train Loss: 0.02480737278372121
Current Learning Rate: 0.0005
Best sklearn masked F1: 0.9596342079482945
Epoch: 98


100%|██████████| 434/434 [00:02<00:00, 156.27it/s]


Average Dev Loss: 0.5294855833879762
Average Dev accuracy: 0.0
Average Dev F1: 0.9546081339872722
Average train Loss: 0.024253755565230777
Current Learning Rate: 0.0005
Best sklearn masked F1: 0.9596342079482945
Epoch: 99


100%|██████████| 434/434 [00:02<00:00, 160.47it/s]

Average Dev Loss: 0.5564461686929542
Average Dev accuracy: 0.0
Average Dev F1: 0.9567873982440075
Average train Loss: 0.024438474007152624
Current Learning Rate: 0.0005
Best sklearn masked F1: 0.9596342079482945


In [86]:
from IPython.display import FileLink
FileLink(r'./best_model.pt')


/kaggle/working/best_model.pt

In [143]:
SAVE_PATH = "best_model.pt"
model = BLSTM(len(vocabulary), embedding_dim=100, hidden_dim=256, output_dim=len(tag2idx), dropout=0.33).to(device)
model.load_state_dict(torch.load(SAVE_PATH))
model.eval()

BLSTM(
  (embedding): Embedding(30292, 100, padding_idx=0)
  (blstm): LSTM(100, 256, batch_first=True, bidirectional=True)
  (dropout1): Dropout(p=0.33, inplace=False)
  (linear1): Linear(in_features=512, out_features=128, bias=True)
  (activation): ELU(alpha=1.0)
  (classifier): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): Tanh()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=9, bias=True)
  )
)

In [144]:
evaluate(model, criterion, dev_loader)

100%|██████████| 434/434 [00:02<00:00, 174.39it/s]

Average Dev Loss: 0.5625147515669688
Average Dev accuracy: 0.0
Average Dev F1: 0.9627304227974504


(0.5625147515669688, 0.9627304227974504)

In [245]:
def build_test_vocab(data):
    # print(data)
    word_counts = Counter(word for _, sentence, _ in data for word in sentence)
    vocabulary = ['<pad>', '<unk>'] + sorted(word_counts)
    word2idx = {word: idx for idx, word in enumerate(vocabulary)}
    return vocabulary, word2idx

_, test_word2idx_untouched = build_test_vocab(train_data+dev_data+test_data)
# print(test_word2idx_untouched)

In [240]:
def make_dev_for_perl_helper(idx2tag, reverse_test_word2idx_untouched, outputs, batch_x_untouched, batch_gold, batch_ind, file):
    acc = 0
    for i in range(outputs.shape[0]):
        sentence_pred = outputs[i]
        one_x = batch_x_untouched[i]
        sentence_index = batch_ind[i]
        goldens = batch_gold[i]
        for j, word_probs in enumerate(sentence_pred):
            word_pred = torch.argmax(word_probs).item()
            # tag = tag2idx[str(word_pred)]
            # print(sentence_index)
            index = sentence_index[j].item()
            # with open("")
            if index == 0:
              break
            word = reverse_test_word2idx_untouched[one_x[j].item()]
            tag = idx2tag[word_pred]
            gold = goldens[j].item()
            gold = idx2tag[gold]

            if index == 1:
              # print('\n')
              file.write('\n')
            file.write(str(index) + ' ' + word + ' ' + gold+ ' ' + tag + '\n')
            # print(index, word, tag)

def make_dev_for_perl(model, idx2tag, reverse_test_word2idx_untouched, dataloader, file_name, device):
  with torch.no_grad():
    with open(file_name, 'w') as file:
      dev_acc = 0.0
      for batch_x, batch_gold, batch_ind, batch_x_untouched in tqdm(dataloader):
        batch_x = batch_x.to(device)
        batch_gold = batch_gold.to(device)
#         print(batch_x.shape)
        outputs = model(batch_x)
        make_dev_for_perl_helper(idx2tag, reverse_test_word2idx_untouched, outputs, batch_x_untouched, batch_gold, batch_ind, file)
        acc = accuracy(outputs, batch_gold)
        dev_acc += acc
    dev_acc /= len(dataloader)
    print(f"Average accuracy: {dev_acc}")

In [241]:
def make_output_file_helper(idx2tag, reverse_test_word2idx_untouched, outputs, batch_x_untouched, batch_ind, file):
    acc = 0
    for i in range(outputs.shape[0]):
        sentence_pred = outputs[i]
        one_x = batch_x_untouched[i]
        sentence_index = batch_ind[i]
        for j, word_probs in enumerate(sentence_pred):
            word_pred = torch.argmax(word_probs).item()
            # tag = tag2idx[str(word_pred)]
            # print(sentence_index)
            index = sentence_index[j].item()
            # with open("")
            if index == 0:
              break
            word = reverse_test_word2idx_untouched[one_x[j].item()]
            tag = idx2tag[word_pred]

            if index == 1:
              # print('\n')
              file.write('\n')
            file.write(str(index) + ' ' + word + ' ' + tag + '\n')
            # print(index, word, tag)

def make_output_file(model, idx2tag, reverse_test_word2idx_untouched, dataloader, file_name, device):
  with torch.no_grad():
    with open(file_name, 'w') as file:
      dev_acc = 0.0
      for batch_x, batch_x_untouched, batch_ind in tqdm(dataloader):
        batch_x = batch_x.to(device)
        outputs = model(batch_x)
        make_output_file_helper(idx2tag, reverse_test_word2idx_untouched, outputs, batch_x_untouched, batch_ind, file)

In [277]:
mode = 'test_out'

if mode == 'test_out':
    mode_data = test_data
elif mode == 'dev_out' or mode == 'dev_perl':
    mode_data = dev_data
    
print_dataset = NERDataset(mode_data, word2idx, tag2idx, mode, test_word2idx_untouched)
print_loader = DataLoader(print_dataset, batch_size=batch_size)

reverse_test_word2idx_untouched = {v: k for k, v in test_word2idx_untouched.items()}
idx2tag = {v: k for k, v in tag2idx.items()}

In [278]:
from IPython.display import FileLink
if mode == 'dev_perl':
    file_name = 'dev1_perl.out'
    make_dev_for_perl(model, idx2tag, reverse_test_word2idx_untouched, print_loader, file_name, device)
    
FileLink(r'dev1_perl.out')

/kaggle/working/dev1_perl.out

In [279]:
if mode == 'dev_out':
    file_name = 'dev1.out'
    make_output_file(model, idx2tag, reverse_test_word2idx_untouched, print_loader, file_name, device)    
FileLink(r'dev1.out')

/kaggle/working/dev1.out

In [280]:
if mode == 'test_out':
    file_name = 'test1.out'
    make_output_file(model, idx2tag, reverse_test_word2idx_untouched, print_loader, file_name, device)    
FileLink(r'test1.out')

100%|██████████| 461/461 [00:04<00:00, 108.68it/s]


/kaggle/working/test1.out

In [128]:
!perl conll03eval < dev1.out

processed 51577 tokens with 5942 phrases; found: 5673 phrases; correct: 4645.
accuracy:  96.06%; precision:  81.88%; recall:  78.17%; FB1:  79.98
              LOC: precision:  90.58%; recall:  85.36%; FB1:  87.89  1731
             MISC: precision:  79.12%; recall:  74.40%; FB1:  76.69  867
              ORG: precision:  73.10%; recall:  73.97%; FB1:  73.54  1357
              PER: precision:  81.43%; recall:  75.95%; FB1:  78.60  1718


In [84]:
embeddings_index = {}
with open('glove.6B.100d', 'r', encoding='utf-8') as f:
    for line in tqdm(f):
        values = line.split()
        word = values[0].lower()
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
        
len(embeddings_index)

400000it [00:09, 41931.32it/s]


400000

In [87]:
def make_weight_matrix(word2idx):
    weights_matrix = np.zeros((len(vocabulary), 100))
    hits = misses = 0
    # Initialize the unk and pad vector randomly using a normal distribution
    unk_weight = np.random.normal(scale=0.8, size=(100,))
    pad_weight = np.random.normal(scale=0.8, size=(100,))
#     pad_weight = np.zeros(100)
    for word, i in word2idx.items():
    #     print(word)
        embedding_vector = embeddings_index.get(word.lower())
        if embedding_vector is not None:
            weights_matrix[i] = embedding_vector
            hits += 1
        else:
            misses += 1
            if word == '<pad>':
                weights_matrix[i] = pad_weight
            else:
                weights_matrix[i] = unk_weight
    print(f"Hits: {hits} Misses: {misses} Hit Ratio: {hits/(hits+misses)}")
    return weights_matrix


weights_matrix = make_weight_matrix(word2idx)       
embedding_layer = nn.Embedding(len(vocabulary), 100)
embedding_layer.weight.data.copy_(torch.from_numpy(weights_matrix))
embedding_layer.weight.requires_grad = True

Hits: 26340 Misses: 3952 Hit Ratio: 0.8695365112901096


In [88]:
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BLSTM(len(vocabulary), embedding_dim=100, hidden_dim=256, output_dim=len(tag2idx), dropout=0.33, pretrained_embedding = embedding_layer).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, weight_decay = 1e-5, eps=1e-08, betas = (0.9, 0.999))
# optimizer = torch.optim.SGD(model.parameters(), lr = 0.05, momentum = 0.65, weight_decay = 1e-3)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9, weight_decay = 1e-3)

# scheduler = MultiStepLR(optimizer, milestones=[3,5,7,9], gamma=0.1)
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.5, max_lr=1.2, step_size_up=20, step_size_down=None, mode='triangular', gamma=1.0)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', min_lr=1, verbose=True)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',factor=0.75, patience = 5, threshold=0.1, verbose=True, min_lr=5e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',factor=0.55, patience = 3, threshold=0.1, verbose=True, min_lr=5e-4)

# scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.5, total_iters=20)

# class_weights = torch.tensor([1,1,1,1,1,1,1,1,0.01], dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=-1, reduction='mean', weight=class_weights)
    

def train(model, train_loader, optimizer, criterion, device, epochs):
    model.train()
    SAVE_PATH = "./best_model_2.pt"
    best_f1 = -1
    for epoch in range(epochs):
      print(f"Epoch: {epoch}")
      train_loss, train_acc, train_f1 = 0.0, 0.0, 0.0
      for batch_x, batch_y in tqdm(train_loader):
          batch_x = batch_x.to(device)
          batch_y = batch_y.to(device)
          outputs = model(batch_x)

          seq_lengths = torch.count_nonzero(batch_x, dim=1).to('cpu')
          packed_y = pack_padded_sequence(batch_y, seq_lengths, batch_first=True, enforce_sorted=False)
          unpacked_y, unpacked_len = pad_packed_sequence(packed_y, batch_first=True, padding_value = -1)
          unpacked_y = unpacked_y.to(device)
            
          loss = criterion(outputs.permute(0, 2, 1), unpacked_y)
          train_loss += loss.item()
  
#           mask = (unpacked_y >= 0) -->costly operation, uncomment to see accuracy, f1
#           acc = accuracy(outputs, batch_y)
#           out_for_f1 = torch.argmax(outputs, dim = -1)
#           f1 = f1_score(out_for_f1[mask].cpu(), unpacked_y[mask].cpu(), average='weighted')
#           train_acc += acc
#           train_f1 += f1

          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          # print(get_lr(optimizer))

      train_loss /= len(train_loader)
#       train_acc /= len(train_loader)
#       train_f1 /= len(train_loader)
      
#       print(f"Average train accuracy: {train_acc}")
#       print(f"Average train f1: {train_f1}")
      val_loss, val_f1 = evaluate(model, criterion, dev_loader)
      if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), SAVE_PATH)
      scheduler.step(val_loss)
      print(f"Average train Loss: {train_loss}")
      print(f"Current Learning Rate: {get_lr(optimizer)}")
      print(f"Best sklearn masked F1: {best_f1}")

    return model

model = train(model, train_loader, optimizer, criterion, device, epochs = 100)


Epoch: 0


100%|██████████| 434/434 [00:02<00:00, 174.71it/s]


Average Dev Loss: 0.7340873273149613
Average Dev accuracy: 0.0
Average Dev F1: 0.7705576421574798
Average train Loss: 1.0544551648954954
Current Learning Rate: 0.01
Best sklearn masked F1: 0.7705576421574798
Epoch: 1


100%|██████████| 434/434 [00:02<00:00, 178.20it/s]


Average Dev Loss: 0.541081804717775
Average Dev accuracy: 0.0
Average Dev F1: 0.7525595739373699
Average train Loss: 0.5337999259681304
Current Learning Rate: 0.01
Best sklearn masked F1: 0.7705576421574798
Epoch: 2


...

Average Dev Loss: 0.3799737037153065
Average Dev accuracy: 0.0
Average Dev F1: 0.9603896379264439
Average train Loss: 0.027420283760502737
Current Learning Rate: 0.0005
Best sklearn masked F1: 0.9667740115298565
Epoch: 98


100%|██████████| 434/434 [00:02<00:00, 158.76it/s]


Average Dev Loss: 0.40372101074054595
Average Dev accuracy: 0.0
Average Dev F1: 0.9632844905972828
Average train Loss: 0.027640321590413866
Current Learning Rate: 0.0005
Best sklearn masked F1: 0.9667740115298565
Epoch: 99


100%|██████████| 434/434 [00:02<00:00, 164.83it/s]

Average Dev Loss: 0.4062223510756608
Average Dev accuracy: 0.0
Average Dev F1: 0.9620557984641228
Average train Loss: 0.02718517045453668
Current Learning Rate: 0.0005
Best sklearn masked F1: 0.9667740115298565


In [140]:
SAVE_PATH = "best_model_2.pt"
model = BLSTM(len(vocabulary), embedding_dim=100, hidden_dim=256, output_dim=len(tag2idx), dropout=0.33).to(device)
model.load_state_dict(torch.load(SAVE_PATH))
model.eval()

BLSTM(
  (embedding): Embedding(30292, 100, padding_idx=0)
  (blstm): LSTM(100, 256, batch_first=True, bidirectional=True)
  (dropout1): Dropout(p=0.33, inplace=False)
  (linear1): Linear(in_features=512, out_features=128, bias=True)
  (activation): ELU(alpha=1.0)
  (classifier): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): Tanh()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=9, bias=True)
  )
)

In [ ]:
mode = 'test_out'

if mode == 'test_out':
    mode_data = test_data
elif mode == 'dev_out' or mode == 'dev_perl':
    mode_data = dev_data
    
print_dataset = NERDataset(mode_data, word2idx, tag2idx, mode, test_word2idx_untouched)
print_loader = DataLoader(print_dataset, batch_size=batch_size)

In [ ]:
from IPython.display import FileLink
if mode == 'dev_perl':
    file_name = 'dev2_perl.out'
    make_dev_for_perl(model, idx2tag, reverse_test_word2idx_untouched, print_loader, file_name, device)
    
FileLink(r'dev2_perl.out')

In [ ]:
if mode == 'dev_out':
    file_name = 'dev2.out'
    make_output_file(model, idx2tag, reverse_test_word2idx_untouched, print_loader, file_name, device)    
FileLink(r'dev2.out')

In [ ]:
if mode == 'test_out':
    file_name = 'test2.out'
    make_output_file(model, idx2tag, reverse_test_word2idx_untouched, print_loader, file_name, device)    
FileLink(r'test2.out')

In [134]:
!perl conll03eval < dev2.out

processed 51577 tokens with 5942 phrases; found: 6190 phrases; correct: 5066.
accuracy:  96.98%; precision:  81.84%; recall:  85.26%; FB1:  83.51
              LOC: precision:  88.91%; recall:  92.54%; FB1:  90.69  1912
             MISC: precision:  70.30%; recall:  73.43%; FB1:  71.83  963
              ORG: precision:  78.05%; recall:  79.27%; FB1:  78.65  1362
              PER: precision:  83.26%; recall:  88.27%; FB1:  85.69  1953
